# N-Gram Models

In [338]:
from math import log, exp, prod
from pprint import pprint

from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize

## Normalizing the text

## Maximum Likelihood Estimation
- The maximum likelihood of a sequence $$w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i$$ is the likelihood of $w_i$ coming after the sequence $$w_{i-n+1},\dots ,w_{i-2},w_{i-1}$$
- Thus,
$$
P(w_i|w_{i-n+1},\dots ,w_{i-2},w_{i-1}) = \frac{
    C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i)
    }{
        \sum_w C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w)
    }
$$
- Now, the number of n-grams starting with $w_{i-n},\dots ,w_{i-2},w_{i-1}$ is the same as the number of times the sequence appears, thus $$  \sum_w C(w_{i-n},\dots ,w_{i-2},w_{i-1},w) = C(w_{i-n},\dots ,w_{i-2},w_{i-1}) $$
- Thus, the MLE becomes
$$
P(w_i|w_{i-n+1},\dots ,w_{i-2},w_{i-1}) = \frac{
    C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i)
    }{
        C(w_{i-n+1},\dots ,w_{i-2},w_{i-1})
    }
$$

In [339]:
class NGramModel:
    def __init__(self, text: str, n: int, keep_punctuation: bool=False, verbose: bool=False):
        self.text = text
        self.n = n
        self.verbose = verbose
        self.keep_punctuation = keep_punctuation
        sentences, words = self._tokenize(text)
        self.n_grams = self._make_n_grams(sentences, n)
        self.n1_grams = self._make_n_grams(sentences, n-1)
        # if self.verbose:
            # pprint(self.n_grams)

    def _tokenize(self, text: str):
        sentences = [
            word_tokenize(sentence)
                for sentence in sent_tokenize(text)
        ]
        sentences = [(
            ['<s>']*(self.n-1) + sentence + ['</s>']*(self.n-1)
        ) for sentence in sentences]
        
        punctutation=['.', ',', '!', '?']
        words = [
            word
                for sentence in sentences
                    for word in sentence
        ]

        if not self.keep_punctuation:
            sentences = [
                [word for word in sentence
                    if word not in punctutation]
                for sentence in sentences
            ]
            words = [
                word for word in words
                    if word not in punctutation
            ]
        return sentences, words

    def _make_n_grams(self, sentences: list[list[str]], n: int) -> list[tuple[str, ...]]:
        n_grams = [
            ng for sentence in sentences
                for ng in ngrams(sentence, n)
        ]
        return n_grams

    def getMLE(self, word: str, prev_words: tuple[str]) -> float:
        if len(prev_words) >= self.n:
            raise ValueError('prev_words should have length n-1')
        if len(prev_words) < self.n-1:
            prev_words = ('<s>',)*(self.n-1-len(prev_words)) + prev_words
        sequence = tuple(prev_words+(word,))
        count_sequence = self.n_grams.count(sequence)
        count_prev = self.n1_grams.count(prev_words)
        if count_prev == 0:
            mle = 0
        else:
            mle = count_sequence/count_prev
        if self.verbose:
            pprint({
                'seq': sequence,
                'cs': count_sequence,
                'prev': prev_words,
                'cp': count_prev,
                'mle': mle
            })
        return mle

    def getSentenceProb(self, sentence: str) -> float:
        tokenized_sentence, _ = self._tokenize(sentence)
        s = tokenized_sentence[0]
        prob = 1
        prob_pairs = [(
            sent,
            tuple(s[i-self.n+1:i])
        ) for i, sent in enumerate(s)][self.n-1:]
        prob = prod([self.getMLE(*pair) for pair in prob_pairs])
        # prob = exp(sum([log(self.getMLE(*pair)) for pair in prob_pairs]))
        return prob

In [330]:
text = 'I am Sam. Sam I am. I do not like green eggs and ham'

In [340]:
ngm = NGramModel(text, 2)
ngm.getSentenceProb('I am Sam')

0.1111111111111111

In [262]:
ngm = NGramModel(text, 2, verbose=True)
ngm.getMLE('Sam', ('<s>',))
ngm.getMLE('I', ('<s>',))
ngm.getMLE('</s>', ('Sam',))
ngm.getMLE('do', ('I',))
ngm.getMLE('am', ('I',))

{'cp': 3,
 'cs': 1,
 'mle': 0.3333333333333333,
 'prev': ('<s>',),
 'seq': ('<s>', 'Sam')}
{'cp': 3,
 'cs': 2,
 'mle': 0.6666666666666666,
 'prev': ('<s>',),
 'seq': ('<s>', 'I')}
{'cp': 2, 'cs': 1, 'mle': 0.5, 'prev': ('Sam',), 'seq': ('Sam', '</s>')}
{'cp': 3,
 'cs': 1,
 'mle': 0.3333333333333333,
 'prev': ('I',),
 'seq': ('I', 'do')}
{'cp': 3,
 'cs': 2,
 'mle': 0.6666666666666666,
 'prev': ('I',),
 'seq': ('I', 'am')}


0.6666666666666666

In [236]:
ngm = NGramModel(text, 2)
ngm.getMLE('Sam', ('<s>',))
ngm.getMLE('I', ('<s>',))
ngm.getMLE('</s>', ('Sam',))
ngm.getMLE('do', ('I',))
ngm.getMLE('am', ('I',))

0.6666666666666666

In [366]:
mod = NGramModel('who are you. who am i. i am you', 2)
mod.getMLE('are', ('who',))

0.5

## N-Grams for the Sherlock Holmes Corpus

In [341]:
sherlock_text = ''
with open('sherlock.txt', 'r') as file:
    sherlock_text = file.read()
sherlock_text = sherlock_text.lower()

In [342]:
shngm = NGramModel(sherlock_text, 2)

In [343]:
shngm.getMLE('holmes', ('sherlock',))

0.9448441247002398

In [361]:
shngm.getMLE('when', ())

0.006805867582925862

In [358]:
shngm.getSentenceProb('what happened')

7.570162194609999e-06

In [364]:
print(shngm.getSentenceProb('from holmes'))
print(shngm.getMLE('holmes', ('from',)))

1.4828968718906952e-06
0.003089598352214212
